# Práctica III Fundamentos de programacion

## Uso de métodos de la clase GestorStock

## Importaciones

In [ ]:
from gestor_stock import GestorStock

## 1) - Instanciar un objeto GestorStock

In [ ]:
stock = GestorStock()

## 2) - Cargar datos.
### El método lee el archivo y carga datos en una lista
### El método devuelve True si ha cargado el archivo o error si no lo consigue cargar

In [ ]:
stock.cargar_datos()

## 3) - Buscar recursivamente un medicamento
### buscamos con el método buscar_recursivo('id del medicamento)
### Si el medicamento existe, el método devuleve una tupla con la una lista con la información del medicamento y el nº de veces
### que se ha llamado recursivamente a la función
### para buscar el medicamento con id 150 tendriamos lo siguiente
### En caso de no existir el medicamento devuelve mensaje con dato no encontrado y el nº de llamadas

In [ ]:
dato=stock.buscar_recursivo('120')
print(dato)

In [ ]:
dato1=stock.buscar_recursivo('200')
print(dato1)

## 3) - Actualizar cantidad.
### El método actualizar cantidad recibe como parámetro un id de medicamento y una cantidad para actualizar stock
### imprime un mensaje del medicamento y cantidad actualizada y salva la información al archivo
### En caso de no exitir el id del medicamento informa que no se ha encontrado
### Devuelve true si se guarda correctamente el fichero y OSError en caso contrario

In [ ]:
stock.actualizar_cantidad('149',33)

In [ ]:
stock.actualizar_cantidad('50','500')

## 4) - Eliminar caducados
### El método eliminar_caducados() compara la fecha de hoy con la que figura en el archivo
### en el caso de que figure una fecha anterior se elimina del stock, informando de que medicamento se elimina
### Por último, guarda los datos actualizados en el archivo
### Devuelve True si el archivo se guarda correctamente, OSError en caso contrario

In [ ]:
stock.eliminar_caducados()

## 5)- Guardar datos
### El método guardar datos no recibe parámetros.
### Si no hay datos que guardar se informa mediante un mensaje
### Devuelve OSError en caso de que no se pueda guardar en archivo y True en el resto de casos


In [ ]:
stock.guardar_datos()

#### Hola Gemini. Dame pruebas unitarias para probar los métodos de clase stock_medicamento que te copio a continuación:

### Respuesta:
####import unittest
import os
from gestor import GestorStock # Asumiendo que tu archivo se llama gestor.py

class TestGestorStock(unittest.TestCase):

    def setUp(self):
        """Configuración previa a cada test: crear un CSV de prueba."""
        self.gestor = GestorStock()
        self.nombre_archivo = "actividad3_stock_medicamentos.csv"
        self.nombre_archivo_salida = "actividad3_stock_medicamentosI.csv"

        # Creamos un archivo de ejemplo para las pruebas
        with open(self.nombre_archivo, 'w', encoding='utf-8') as f:
            f.write('id_medicamento;nombre;cantidad;fecha_caducidad\n')
            f.write('1;Paracetamol;50;2026-01-01\n')
            f.write('2;Ibuprofeno;20;2023-01-01\n') # Ya caducado
            f.write('3;Aspirina;10;2025-12-31\n')

    def tearDown(self):
        """Limpieza después de cada test: borrar archivos temporales."""
        if os.path.exists(self.nombre_archivo):
            os.remove(self.nombre_archivo)
        if os.path.exists(self.nombre_archivo_salida):
            os.remove(self.nombre_archivo_salida)

    def test_cargar_datos_exitoso(self):
        """Verifica que los datos se cargan correctamente en la lista."""
        resultado = self.gestor.cargar_datos()
        self.assertTrue(resultado)
        self.assertEqual(len(self.gestor.datos), 3)
        self.assertEqual(self.gestor.datos[0][1], "Paracetamol")

    def test_cargar_datos_archivo_no_existe(self):
        """Verifica que lanza FileNotFoundError si el archivo falta."""
        os.remove(self.nombre_archivo)
        with self.assertRaises(FileNotFoundError):
            self.gestor.cargar_datos()

    def test_buscar_recursivo_encontrado(self):
        """Verifica la búsqueda de un ID existente."""
        resultado, pasos = self.gestor.buscar_recursivo("2")
        self.assertEqual(resultado[1], "Ibuprofeno")
        self.assertGreaterEqual(pasos, 1)
        # Verificar que limpia los datos tras la búsqueda
        self.assertEqual(len(self.gestor.datos), 0)

    def test_buscar_recursivo_no_encontrado(self):
        """Verifica el comportamiento cuando el ID no existe."""
        resultado, pasos = self.gestor.buscar_recursivo("99")
        self.assertEqual(resultado, "Dato no encontrado")

    def test_actualizar_cantidad(self):
        """Verifica que se actualiza la cantidad de un medicamento."""
        self.gestor.actualizar_cantidad("1", 100)

        # Para verificarlo, cargamos el archivo de salida que genera tu método
        self.gestor.cargar_datos() # Esto cargará el original, pero el método guarda en ...I.csv
        # Ajustamos para leer el archivo de salida
        with open(self.nombre_archivo_salida, 'r', encoding='utf-8') as f:
            lineas = f.readlines()
            self.assertIn("1;Paracetamol;100;2026-01-01\n", lineas)

    def test_eliminar_caducados(self):
        """Verifica que se eliminan los medicamentos con fecha anterior a hoy."""
        # El ID 2 (Ibuprofeno) pusimos fecha 2023-01-01, debería borrarse
        self.gestor.eliminar_caducados()

        # Comprobar que en la lista actual ya no está el caducado
        ids_restantes = [m[0] for m in self.gestor.datos]
        self.assertNotIn("2", ids_restantes)
        self.assertIn("1", ids_restantes)

if __name__ == '__main__':
    unittest.main()